In [ ]:
!pip install spacy
!pip install newsapi-python

In [ ]:
!python -m spacy download en_core_web_lg



In [78]:
import spacy
import pandas as pd
from newsapi import NewsApiClient
from queue import PriorityQueue
nlp = spacy.load("en_core_web_sm")
newsapi = NewsApiClient (api_key='efebc02ff1404817b34af91448a497dd')

articles = newsapi.get_everything(q='coronavirus', language='en',from_param='2021-02-19', to='2021-03-19', sort_by='relevancy')

article=articles["articles"]
dados=[]
for i in article:
  dados.append({'title':i["title"],'date':i['publishedAt'],'desc':i['description'],'content':i['content']})

df = pd.DataFrame(dados)
df = df.dropna()

pos_tag=["VERB","NOUN","PRON"]
result=[]

for ar in article:
#this loop stores the 5 keywords of each article
  keywords=[]
  common={}
  doc=nlp(ar["content"]+ar["description"]+ar["title"])
  for token in doc:
    #get all the words with Verb,noun, and pronoun
      if (token.text in nlp.Defaults.stop_words):
        continue
      if (token.pos_ in pos_tag):
        keywords.append(token.text)
  #count the number of times the word appeared
  for i in keywords:
    if i in common:
      common[i]+=1
    
    else: 
      common[i]=1
  #put the keywords in priority queue so we know which words appear the most  
  find_common=PriorityQueue()

  for x,y in common.items():
    find_common.put((y,x))
  
  #get the most common five words
  iterator=find_common.qsize()-1
  ender=0
  if(iterator>=6):
    ender=iterator-4
    for i in range(ender):
      find_common.get()

  final_common=[]
  while not find_common.empty():
    item=find_common.get()
    final_common.append(item[1])
  #put the keywords in result.
  result.append(final_common)
df["keywords"]=result

df.head(20)




,title,date,desc,content,keywords
0,Coronavirus Briefing: What Happened Today,2021-03-12T23:12:19Z,An optimistic view of the weeks ahead.,What do you think the next few months look lik...,"[weeks, months, think, What, going]"
1,"Your N.B.A. Coronavirus Questions, Answered",2021-03-11T22:49:26Z,Can fans attend all games? Will players be vac...,Why havent I seen Wayne Knight in the ads for ...,"[sorry, states, vaccinated, weekend, I]"
2,Coronavirus Briefing: What Happened Today,2021-03-11T23:32:45Z,"Israel’s reopening, and vaccine doses in limbo.","How is the Green Passport system working, for ...","[restaurants, sit, system, vaccine, working]"
3,Coronavirus Briefing: What Happened Today,2021-03-16T22:21:05Z,The U.S. political divide is affecting its vac...,Similar laborious efforts may have to be scale...,"[track, tries, tryp, vaccine, way]"
4,Coronavirus Briefing: What Happened Today,2021-03-09T23:04:24Z,People with underlying conditions are begging ...,But medical ethicists say the list is misleadi...,"[tha, underlying, vaccine, conditions, list]"
5,"As Coronavirus Variants Spread, the US Struggl...",2021-02-19T12:00:00Z,The nation is a sequencing superpower. But wit...,"Its not a unique quirk, but like many other Co...","[tests, threat, track, viruss, snippets]"
6,Coronavirus Briefing: What Happened Today,2021-02-24T22:05:31Z,Johnson & Johnson’s vaccine was found to provi...,"The global vaccine-sharing program Covax, whic...","[sharing, shipment, today, world, vaccine]"
7,Coronavirus Briefing: What Happened Today,2021-03-19T22:11:40Z,More states said they will open vaccine appoin...,Artists and institutions are struggling after ...,"[switc, turmoil, uses, vaccine, year]"
8,Coronavirus Briefing: What Happened Today,2021-03-10T23:21:20Z,What the $1.9 trillion coronavirus relief plan...,"Community health centers, which Abby said act ...","[pandemic, plan, relief, said, health]"
9,Coronavirus Briefing: What Happened Today,2021-03-15T22:25:20Z,Europe is struggling with a third wave.,"A year into the pandemic, The Times collected ...","[wave, ways, year, pandemic, readers]"
